# Caltech Pedestrian Annotation

The Caltech Pedestrians dataset (http://www.vision.caltech.edu/Image_Datasets/CaltechPedestrians/) is separated into images and annotations.

Using a third-party converter (https://github.com/mitmul/caltech-pedestrian-dataset-converter), I have the extracted the annotations into a massive JSON file. I will be exploring the annotations in order to get a better feel for the data, and to pick a subset of images I would like to train on.

In particular, the annotations are stored in `.vbb` format, which stands for "video bounding box." Someone on the internet gives the following explanation of the format:

```
A video bounding box (vbb) annotation stores bounding boxes (bbs) for
objects of interest. The primary difference from a static annotation is
that each object can exist for multiple frames, ie, a vbb annotation not
only provides the locations of objects but also tracking information. A
vbb annotation A is simply a Matlab struct. It contains data per object
(such as a string label) and data per object per frame (such as a bb).
Each object is identified with a unique integer id.

Data per object (indexed by integer id) includes the following fields:
 init - 0/1 value indicating whether object w given id exists
 lbl  - a string label describing object type (eg: 'pedestrian')
 str  - the first frame in which object appears (1 indexed)
 end  - the last frame in which object appears (1 indexed)
 hide - 0/1 value indicating object is 'hidden' (used during labeling)

Data per object per frame (indexed by frame and id) includes:
 pos  - [l t w h]: bb indicating predicted object extent
 posv - [l t w h]: bb indicating visible region (may be [0 0 0 0])
 occl - 0/1 value indicating if bb is occluded
 lock - 0/1 value indicating bb is 'locked' (used during labeling)
```

In [4]:
import json

with open('../CaltechPedestrians/data/annotations.json') as f:
    raw = json.load(f)

In [17]:
for k, v in raw['set00']['V000'].items():
    print(k)

nFrame
maxObj
log
logLen
altered
frames


In [29]:
raw['set00']['V000']['frames']['88']

[{'end': 371,
  'hide': 0,
  'id': 2,
  'init': 1,
  'lbl': 'person',
  'lock': 0,
  'occl': 0,
  'pos': [225.3247422320086,
   136.01476564957534,
   5.606859337317815,
   11.767162105483848],
  'posv': [0, 0, 0, 0],
  'str': 69}]